### 2020 Data

In [1]:
import pandas as pd
import numpy as np

In [ ]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020" # fetching all the 2020 movies from wikipedia
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [12]:
df = pd.concat([df1,df2,df3,df4],ignore_index=True) #concat all the dataframes in a single for preprocessing
df

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN
4,J A N U A R Y,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International ...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
...,...,...,...,...,...,...
271,D E C E M B E R,25,We Can Be Heroes,Netflix / Troublemaker Studios / Double R Prod...,Robert Rodriguez (director/screenplay); Priyan...,[244]
272,D E C E M B E R,25,News of the World,Universal Pictures / Playtone / Perfect World ...,Paul Greengrass (director/screenplay); Luke Da...,[245]
273,D E C E M B E R,25,One Night in Miami...,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,[246]
274,D E C E M B E R,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,[247]


In [15]:
df.columns

Index(['Opening', 'Opening.1', 'Title', 'Production company', 'Cast and crew',
       'Ref.'],
      dtype='object')

#### As it is observed in the above table, we do not have the genre column so we will get the genre data from IMDb. You will need to create an API key in order to access the API key.

### pip install tmdbv3api

In [16]:
#get your API Key here: https://www.themoviedb.org/settings/api
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()

api = 'cf1e5b9d8cf31e19913dd6bc256abb2a'
tmdb.api_key=api # insert your api key here


In [22]:
import requests

from tmdbv3api import Movie

tmdb_movie = Movie()

def get_genre(x):
    genres = []
    
    try:
        # Search for the movie title
        result = tmdb_movie.search(x)
        
        # Check if any results were found
        if not result:
            return 'No results found'

        # Get the movie ID from the first result
        movie_id = result[0].id
        
        # Fetch movie details using the movie ID
        response = requests.get(f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}')
        data_json = response.json()
        
        # Check if the movie has genres
        if 'genres' in data_json and data_json['genres']:
            for genre in data_json['genres']:
                genres.append(genre['name'])
            return ' '.join(genres)  # Return genres as a space-separated string
        else:
            return np.nan
    
    except Exception as e:
        # Handle any exceptions (e.g., connection issues, API errors)
        return f'Error: {str(e)}'

# Example usage to map genres to the DataFrame
df['genres'] = df['Title'].map(lambda x: get_genre(x))


In [23]:
# df['genres']= df['Title'].map(lambda x : get_genre(x))

df.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.,genres
0,J A N U A R Y,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2],Horror Mystery Thriller
1,J A N U A R Y,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3],Horror Science Fiction Action Adventure
2,J A N U A R Y,10,Like a Boss,Paramount Pictures / Artists First,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4],Comedy
3,J A N U A R Y,10,Three Christs,IFC Films,Jon Avnet (director/screenplay); Eric Nazarian...,NaN,Drama
4,J A N U A R Y,10,Inherit the Viper,Lionsgate / Barry Films / Tycor International ...,Anthony Jerjen (director); Andrew Crabtree (sc...,[5],Crime Thriller Drama


In [24]:
import re


def extract_info(text):
    # Initialize the director to None
    director_name = None
    # Initialize the actors to None
    actor_1, actor_2, actor_3 = None, None, None
    
    # Check if the text contains a director's name and role(s)
    director_match = re.search(r'([a-zA-Z\s]+) \(([^)]+)\)', text)  # Extract name and roles inside parentheses
    if director_match:
        name = director_match.group(1).strip()  # Extract the name (e.g., Michael Moore)
        roles = director_match.group(2).strip()  # Extract the roles (e.g., director/screenplay/narrator)
        
        # If 'director' is in the roles, assign only the name to the director column
        if 'director' in roles:
            director_name = name  # Only assign the name to the director column if 'director' is present in the roles
            
            # Remove the director's name from the actors list to avoid duplication
            text = text.replace(name + " (" + roles + ")", "")  # Remove the entire director's entry

    # Extract actors from the cast and crew field (after the last semicolon)
    actors = re.split(r';', text)[-1].strip().split(', ')  # Split by semicolon and take actors from the last part
    
    # Remove the director from the actors list (if it was listed there)
    actors = [actor for actor in actors if actor != director_name]
    
    # Assign actors if they exist
    actor_1 = actors[0] if len(actors) > 0 else None
    actor_2 = actors[1] if len(actors) > 1 else None
    actor_3 = actors[2] if len(actors) > 2 else None

    # Return the extracted information in a pandas Series
    return pd.Series([director_name, actor_1, actor_2, actor_3])



In [25]:
df[['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name']] = df['Cast and crew'].apply(extract_info)


In [26]:
df=df.rename(columns={'Title':'movie_title'})
df.sample(2)

,Opening,Opening.1,movie_title,Production company,Cast and crew,Ref.,genres,director_name,actor_1_name,actor_2_name,actor_3_name
19,F E B R U A R Y,7,The Lodge,Neon / FilmNation Entertainment / Hammer Films,"Veronika Franz, Severin Fiala (directors/scree...",[20],Horror Drama Mystery Thriller,Severin Fiala,Riley Keough,Jaeden Martell,Lia McHugh
136,A U G U S T,21,Cut Throat City,Well Go USA Entertainment,RZA (director); Paul Cuschieri (screenplay); S...,[130],Action Crime Drama Thriller,RZA,Shameik Moore,T.I.,Demetrius Shipp Jr.


In [27]:
new_df20 = df[['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]
new_df20.sample(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
256,Tara Miele,Sienna Miller,Diego Luna,Beth Grant,Romance Drama,Wander Darkly
200,Evan Morgan,Adam Brody,Sophie Nélisse,Tzi Ma,Comedy Drama Mystery,The Kid Detective


In [29]:
new_df20.isna().sum()

director_name     4
actor_1_name      0
actor_2_name      7
actor_3_name     29
genres            0
movie_title       0
dtype: int64

In [30]:
new_df20.fillna('unknown',inplace=True)

C:\Users\ARMS\AppData\Local\Temp\ipykernel_18216\1337849485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20.fillna('unknown',inplace=True)


In [31]:
new_df20.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
dtype: int64

In [33]:
new_df20['movie_title']= new_df20['movie_title'].str.lower()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_18216\1593271720.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['movie_title']= new_df20['movie_title'].str.lower()


In [34]:

new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']
new_df20.head()

C:\Users\ARMS\AppData\Local\Temp\ipykernel_18216\2214040608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df20['comb'] = new_df20['actor_1_name'] + ' ' + new_df20['actor_2_name'] + ' '+ new_df20['actor_3_name'] + ' '+ new_df20['director_name'] +' ' + new_df20['genres']


,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho,Horror Mystery Thriller,the grudge,Andrea Riseborough Demián Bichir John Cho Nico...
1,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick,Horror Science Fiction Action Adventure,underwater,Kristen Stewart Vincent Cassel Jessica Henwick...
2,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek,Comedy,like a boss,Tiffany Haddish Rose Byrne Salma Hayek Miguel ...
3,Jon Avnet,Richard Gere,Peter Dinklage,Walton Goggins,Drama,three christs,Richard Gere Peter Dinklage Walton Goggins Jon...
4,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs,Crime Thriller Drama,inherit the viper,Josh Hartnett Margarita Levieva Chandler Riggs...


In [43]:
old = pd.read_csv(r'D:\Data Science Pojects\Netflix Recommendation System\data_2018\final_data.csv')

old.sample(2)

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
752,Tom Shadyac,Joe Morton,Susanna Thompson,Matt Craven,dragonfly,Drama Fantasy Mystery Romance Thriller,Joe Morton Susanna Thompson Matt Craven Tom Sh...
2680,John Singleton,Janet Jackson,Khandi Alexander,Maya Angelou,poetic justice,Drama Romance,Janet Jackson Khandi Alexander Maya Angelou Jo...


In [44]:
final = pd.concat([old,new_df20],ignore_index=True)
final

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,pirates of the caribbean: at world's end,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,spectre,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,the dark knight rises,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,star wars: episode vii - the force awakens ...,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6147,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,we can be heroes,Family Action Fantasy Comedy,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
6148,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,news of the world,Drama Western Adventure,Tom Hanks Helena Zengel unknown Paul Greengras...
6149,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,one night in miami...,Drama,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
6150,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,promising young woman,Thriller Crime Drama,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [45]:
final.isna().sum()

director_name    0
actor_1_name     3
actor_2_name     1
actor_3_name     0
movie_title      0
genres           0
comb             0
dtype: int64

In [46]:
final.dropna(inplace=True)

In [47]:
final.drop_duplicates(keep='last',inplace=True)

final

,director_name,actor_1_name,actor_2_name,actor_3_name,movie_title,genres,comb
0,James Cameron,CCH Pounder,Joel David Moore,Wes Studi,avatar,Action Adventure Fantasy Sci-Fi,CCH Pounder Joel David Moore Wes Studi James C...
1,Gore Verbinski,Johnny Depp,Orlando Bloom,Jack Davenport,pirates of the caribbean: at world's end,Action Adventure Fantasy,Johnny Depp Orlando Bloom Jack Davenport Gore ...
2,Sam Mendes,Christoph Waltz,Rory Kinnear,Stephanie Sigman,spectre,Action Adventure Thriller,Christoph Waltz Rory Kinnear Stephanie Sigman ...
3,Christopher Nolan,Tom Hardy,Christian Bale,Joseph Gordon-Levitt,the dark knight rises,Action Thriller,Tom Hardy Christian Bale Joseph Gordon-Levitt ...
4,Doug Walker,Doug Walker,Rob Walker,unknown,star wars: episode vii - the force awakens ...,Documentary,Doug Walker Rob Walker unknown Doug Walker Doc...
...,...,...,...,...,...,...,...
6147,Robert Rodriguez,Priyanka Chopra Jonas,Pedro Pascal,YaYa Gosselin,we can be heroes,Family Action Fantasy Comedy,Priyanka Chopra Jonas Pedro Pascal YaYa Gossel...
6148,Paul Greengrass,Tom Hanks,Helena Zengel,unknown,news of the world,Drama Western Adventure,Tom Hanks Helena Zengel unknown Paul Greengras...
6149,Regina King,Kingsley Ben-Adir,Eli Goree,Aldis Hodge,one night in miami...,Drama,Kingsley Ben-Adir Eli Goree Aldis Hodge Regina...
6150,Emerald Fennell,Carey Mulligan,Bo Burnham,Alison Brie,promising young woman,Thriller Crime Drama,Carey Mulligan Bo Burnham Alison Brie Emerald ...


In [48]:
final.to_csv('main_data.csv',index=False)